In [1]:
from IPython.core.display import display, HTML
import os
from pathlib import Path
from cmath import inf
import re, os, csv, pathlib, ast, os.path
import pandas as pd
from statistics import mean, variance
import json
from collections import Counter
import numpy as np

display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', None)


<ipython-input-1-4c7d4e88c7a8>:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
def last_name(pth):
    return pathlib.PurePath(pth).name


def get_dirs(pth):
    return [last_name(f.name) for f in os.scandir(pth) if f.is_dir()]


def get_each_run(lne):
    # print(lne.strip())
    # print(re.findall(": (.+)", lne))
    # print(re.findall(": (.+)", lne)[0])
    # print(list(map(float, re.findall(": (.+)", lne)[0].split(" "))))

    return list(map(float, re.findall(": (.+)", lne)[0].split(" ")))


def get_test_file_from_stats_path(run_number, stats_file_opened):
    return stats_file_opened.name[:-9] + "run_{}/test_infer_log.txt".format(run_number)


# def get_test(stats_file_path):
#     return stats_file_path[:-9] + "run_1/output/test_out.txt"


def WER_test_file(test_file):
    try:
        txt_file = open(test_file, "r")
        lines = txt_file.readlines()
        matched = ""
        for line in lines:
            if "==========>>>>>>Evaluation Greedy WER: " in line:
                txt_file.close()
                return float(line.rstrip().split(": ")[1])
    except:
        txt_file.close()
        print("weiowdnio")
        return inf


def get_eta(func, eta):
    return "-n:" + str(float(eta[4:]))


def accent_distribution(json_path):
    file = open(json_path, 'r')
    paths = [json.loads(sample)["accent"] for sample in file.readlines()]
    counts = Counter(paths)
    return dict(counts.most_common())

def time_fraction(json_parent, speaker):
    total_duration, domain_duration = 0, 0
    for i in range(1,4):
        json_path = "{}/run_{}/train.json".format(json_parent, i)
        file = open(json_path, 'r')
        samples = [sample for sample in file.readlines()]
        total_duration += sum([json.loads(sample)['duration'] for sample in samples])
        domain_duration += sum([json.loads(sample)['duration'] for sample in samples if speaker == json.loads(sample)['accent']])
#         print(speaker, json.loads(samples[0])['audio_filepath'])
    total_duration/=3
    domain_duration/=3
    return "{:.1f}/{:.1f}".format(domain_duration, total_duration)

def sample_fraction(json_parent, speaker):
    # print(speaker)
    total, domain_counts = 0, 0
    for i in range(1,4):
        json_path = "{}/run_{}/train.json".format(json_parent, i)
        file = open(json_path, 'r')
        lines = [line for line in file.readlines()]
        total += len(lines)
        domain_counts += len([json.loads(sample) for sample in lines if speaker == json.loads(sample)['accent']])
    total/=3
    domain_counts/=3
    return "{:.1f}/{:.1f}".format(domain_counts, total)



In [3]:
def get_target(path):
    try:
        search_pattern = r"/target_\d*/"
        matched_string = re.search(search_pattern, path).group()
        new_path = re.sub(search_pattern, os.path.sep, path)
        target = matched_string.split('_')[1][:-1]
        return new_path, target
    except:
        return path, "-"

In [4]:
def parse_two_stage(path, dct):
    pass

In [5]:
def parse_within(path, dct):
    assert(path.split(os.path.sep)[0] == "within")
    path = path.replace(f"{path.split(os.path.sep)[0]}/", "")

    dct["budget_b1"] = path.split(os.path.sep)[0].split('_')[-1]
    path = path.replace(f"{path.split(os.path.sep)[0]}/", "")



    assert(path.split(os.path.sep)[0] in ["error_model", "random", "error_model_orig_transc", "true_wer"])
    dct["method"] += path.split(os.path.sep)[0]
    # print(path)
    path = path.replace(f"{path.split(os.path.sep)[0]}/", "")


    # print(path)
    assert(path == "")
    return

In [6]:
def fill_results(orig_path, dct, speaker):
    try:
        dct["time_fraction"] = time_fraction(orig_path, speaker)
        dct["sample_fraction"] = sample_fraction(orig_path, speaker)
        
        dct["speakers"] = accent_distribution(os.path.join(orig_path, "run_1/train.json"))
        
        wer_list = []
        for run in range(1,4):
            test_file_path = os.path.join(orig_path, f"run_{run}", "test_infer_log.txt")
            wer_list.append(WER_test_file(test_file_path))
        
#         print(wer_list, len(wer_list))
            
        mean = np.nanmean(wer_list)
        var = np.nanvar(wer_list)
        dct["WER-mean"] = round(mean, 2)
        dct["WER-stdev"] = round(var**0.5, 2)
        for run in range(1, 4):
            dct[f"WER-r{run}"] = wer_list[run - 1]
        
    except:
        print("Unable to obtain wers from: ", orig_path, dct["method"], dct["budget_b1"])

In [7]:
# sample_path = 'Error-Driven-ASR-Personalization/CMU_expts/speaker/hindi/manifests/TSS_output/all/budget_100/target_50/FL1MI/eta_1.0/euclidean/39/stats.txt'
# CMU_expts/speaker_without/ABA/manifests/TSS_output/all/budget_100/target_50/FL1MI/eta_1.0/euclidean/39/run_1/
# budget = 100


# csv_name = "mod_report_{}_{}.csv".format(budget, target)

# df = pd.DataFrame(columns=cols)

speakers = [
    "african",
    "australia",
    "bermuda",
    "canada",
    "england",
    "hongkong",
    "indian",
    "ireland",
    "malaysia",
    "philippines",
    "scotland",
    "southatlandtic",
    "us",
    "wales",
]

expt_results = []
os_sep = os.path.sep
submod_fxns = ("FL1MI", "FL2MI", "GCMI", "LogDMI")

for speaker in speakers:
    print("Extracting experiments from {}".format(speaker))
    base_dir = os.path.join(".", f"{speaker}", "all")
    all_paths = list(set([f"{os.path.sep}".join(str(p).split(os.path.sep)[:-2]) for p in Path(base_dir).rglob("*/test_infer_log.txt")]))
    for path in all_paths:
        dct = {}
        orig_path = str(path) + "/"
        path = (str(path) + "/").replace(f"{speaker}/all/", "")
        dct["speaker"] = speaker
        
        if path.startswith("budget_"):
            dct["method"] = "Two_stage-"
            parse_two_stage(path, dct) 
            continue
        elif path.startswith("within"):
            dct["method"] = "Acc-within-"
            parse_within(path, dct)

        

        fill_results(orig_path, dct, speaker)
        expt_results.append(dct)

    print("End of {}".format(speaker))
#         else:
#             if path.startswith("dim_phoneme_gains"):
#                 dct["method"] = "phone_decay-"
#                 path = path.replace(f"{path.split(os.path.sep)[0]}/", "")
#                 tau_str = path.split(os.path.sep)[0]
#                 tau_val = tau_str.split('_')[-1]
#                 dct["method"] += tau_val
#                 path = path.replace(f"{path.split(os.path.sep)[0]}/", "")
# #             print(path)
#             else:
#                 dct["method"] = path.split(os.path.sep)[0]
#                 path = path.replace(f"{dct['method']}/", "")
#             dct["other_accents"] = replace_with_short_forms(path.split(os.path.sep)[0])
#             path = path.replace(f"{path.split(os.path.sep)[0]}/", "")
#         assert(path.startswith("budget_"))
#         dct["budget_b1"] = path.split(os.path.sep)[0].replace("budget_", "")
#         path = path.replace(f"budget_{dct['budget_b1']}/", "")
# #         print(path, dct)
        
        
#         if path.startswith("target"):
#             dct["target"] = path.split(os.path.sep)[0].replace("target_", "")
#             path = path.replace(f"target_{dct['target']}/", "")
#         fxn = path.split(os.path.sep)[0]
#         if fxn in submod_fxns:
#             dct["fxn"] = fxn
# #             print(fxn)
#             dct["etaScale"] = "default(1.0)"
            
#         else:
#             ls = fxn.split("_")
#             assert(len(ls) == 3)
#             assert(ls[1] == "etaScale")
#             assert(ls[0] in submod_fxns)
#             dct["fxn"] = ls[0]
#             dct["etaScale"] = ls[2]
# #             print(ls[2])
#         path = path.replace(f"{fxn}/", "")
        
#         if path.startswith("39/"):
#             dct["accent_features"] = "39"
#             path = path.replace(f"{dct['accent_features']}/", "")
#         else:
#             assert(path.startswith("accent_"))
#             dct["accent_features"] = path.split(os.path.sep)[0].replace("accent_", "")
#             path = path.replace(f"accent_{dct['accent_features']}/", "")
        
#         if path.startswith("budget_"):
#             # print(path)
#             dct["budget_b2"] = path.split(os.path.sep)[0].replace("budget_", "")
#             path = path.replace(f"budget_{dct['budget_b2']}/", "")
#             # print(path)
#             dct["method"] = path.split(os.path.sep)[0] + "(stage2)"
#             # print(dct["method"])
#         else:
#             assert(path.startswith("content_"))
#             dct["content_features"] = path.split(os.path.sep)[0].replace("content_", "")
#             path = path.replace(f"content_{dct['content_features']}/", "")
#     #         print(path, dct)
#             if path.startswith("phoneme_"):
#                 path = path.replace(f"{path.split(os.path.sep)[0]}/", "")
            
#             kernels = path.split(os.path.sep)[0].replace("kernel_", "")
#             kernels_ls = "; ".join(kernels.split("_"))
#             dct["kernels"] = kernels_ls
#             path = path.replace(f"kernel_{kernels}/", "")
#             assert(path.startswith("accent_"))
#             dct["accent_sim"] = path.split(os.path.sep)[0].replace("accent_", "")
#             path = path.replace(f"accent_{dct['accent_sim']}/", "")
            
#             assert(path.startswith("content_"))
#             dct["content_sim"] = path.split(os.path.sep)[0].replace("content_", "")
#             path = path.replace(f"content_{dct['content_sim']}/", "")
        
        


Extracting experiments from african
End of african
Extracting experiments from australia
End of australia
Extracting experiments from bermuda
End of bermuda
Extracting experiments from canada
End of canada
Extracting experiments from england
End of england
Extracting experiments from hongkong
End of hongkong
Extracting experiments from indian
End of indian
Extracting experiments from ireland
End of ireland
Extracting experiments from malaysia
End of malaysia
Extracting experiments from philippines
End of philippines
Extracting experiments from scotland
End of scotland
Extracting experiments from southatlandtic
End of southatlandtic
Extracting experiments from us
End of us
Extracting experiments from wales
End of wales


In [8]:
cols = [
    "speaker",
    "other_accents",
    "budget_b1",
    "target",
    "fxn",
    "method",
    "budget_b2",
    "etaScale",
    "accent_features",
    "content_features",
    "accent_sim",
    "content_sim",
    "kernels",
    "time_fraction",
    "sample_fraction",
    "WER-r1",
    "WER-r2",
    "WER-r3",
    "WER-mean",
    "WER-stdev",
    "speakers",
]

df = pd.DataFrame(expt_results, columns=cols)

In [9]:
df = df.sort_values(["speaker", "method"])

In [10]:
df

,speaker,other_accents,budget_b1,target,fxn,method,budget_b2,etaScale,accent_features,content_features,...,content_sim,kernels,time_fraction,sample_fraction,WER-r1,WER-r2,WER-r3,WER-mean,WER-stdev,speakers
2,african,NaN,250,NaN,NaN,Acc-within-error_model,NaN,NaN,NaN,NaN,...,NaN,NaN,1235.0/1235.0,223.7/223.7,22.06,22.66,22.33,22.35,0.25,{'african': 222}
1,african,NaN,250,NaN,NaN,Acc-within-error_model_orig_transc,NaN,NaN,NaN,NaN,...,NaN,NaN,1235.3/1235.3,224.7/224.7,23.34,22.02,22.06,22.47,0.61,{'african': 219}
0,african,NaN,250,NaN,NaN,Acc-within-random,NaN,NaN,NaN,NaN,...,NaN,NaN,1223.8/1223.8,247.7/247.7,21.80,22.66,23.11,22.52,0.54,{'african': 250}
3,african,NaN,250,NaN,NaN,Acc-within-true_wer,NaN,NaN,NaN,NaN,...,NaN,NaN,1232.0/1232.0,235.0/235.0,22.04,22.19,22.29,22.17,0.10,{'african': 235}
4,australia,NaN,250,NaN,NaN,Acc-within-error_model,NaN,NaN,NaN,NaN,...,NaN,NaN,1232.6/1232.6,226.0/226.0,26.06,26.13,25.87,26.02,0.11,{'australia': 220}
5,australia,NaN,250,NaN,NaN,Acc-within-error_model_orig_transc,NaN,NaN,NaN,NaN,...,NaN,NaN,1232.5/1232.5,225.0/225.0,25.85,26.10,26.00,25.98,0.10,{'australia': 213}
7,australia,NaN,250,NaN,NaN,Acc-within-random,NaN,NaN,NaN,NaN,...,NaN,NaN,1224.0/1224.0,249.0/249.0,26.08,26.36,25.39,25.94,0.41,{'australia': 246}
6,australia,NaN,250,NaN,NaN,Acc-within-true_wer,NaN,NaN,NaN,NaN,...,NaN,NaN,1230.1/1230.1,244.0/244.0,25.28,25.14,25.46,25.29,0.13,{'australia': 244}
9,canada,NaN,250,NaN,NaN,Acc-within-error_model,NaN,NaN,NaN,NaN,...,NaN,NaN,1231.2/1231.2,215.7/215.7,17.21,16.68,16.57,16.82,0.28,{'canada': 217}
8,canada,NaN,250,NaN,NaN,Acc-within-error_model_orig_transc,NaN,NaN,NaN,NaN,...,NaN,NaN,1233.5/1233.5,217.0/217.0,16.45,16.75,16.12,16.44,0.26,{'canada': 217}


In [12]:
sorted_df = df.sort_values(["speaker", "fxn", "method"])
sorted_df.to_csv("mcv_250_within.csv", index=False)

In [15]:
mask = (df["method"].str.startswith("phone_decay"))
# mask = (((df["method"].str.endswith("random(stage2)")) | (df["method"].str.startswith("uniform"))) & (df["budget_b1"] == "3500") & (df["budget_b2"] == "150")) 
filtered_df = df.loc[mask]
# filtered_df

In [16]:
filtered_df = filtered_df.sort_values(["other_accents", "fxn", "speaker"])
filtered_df

,speaker,other_accents,budget_b1,target,fxn,method,budget_b2,etaScale,accent_features,content_features,...,content_sim,kernels,time_fraction,sample_fraction,WER-r1,WER-r2,WER-r3,WER-mean,WER-stdev,speakers
5,assamese[F],assamese[F]=1,150,20,FL2MI,phone_decay-1.0,NaN,1.0,39,tf_idf_2gram,...,cosine,g=accent; gq=accent; qq=accent,679.5/734.2,111.0/116.0,20.05,20.33,19.88,20.09,0.19,"{'ass': 111, 'tam': 4, 'guj': 1}"
8,assamese[F],assamese[F]=1,150,20,FL2MI,phone_decay-0.2,NaN,1.0,39,tf_idf_2gram,...,cosine,g=accent; gq=accent; qq=accent,465.3/733.3,76.0/103.0,20.52,20.61,20.12,20.42,0.21,"{'ass': 76, 'tam': 20, 'guj': 5, 'raj': 1, 'ka..."
16,assamese[F],assamese[F]=1,150,20,FL2MI,phone_decay-0.1,NaN,1.0,39,tf_idf_2gram,...,cosine,g=accent; gq=accent; qq=accent,199.3/737.5,31.0/87.0,22.86,22.74,22.81,22.80,0.05,"{'tam': 33, 'ass': 31, 'guj': 14, 'kan': 3, 'r..."
31,assamese[F],assamese[F]=1,150,20,FL2MI,phone_decay-0.3,NaN,1.0,39,tf_idf_2gram,...,cosine,g=accent; gq=accent; qq=accent,553.9/733.9,88.0/106.0,19.66,19.83,20.28,19.92,0.26,"{'ass': 88, 'tam': 14, 'guj': 3, 'kan': 1}"
38,assamese[F],assamese[F]=1,150,20,FL2MI,phone_decay-0.4,NaN,1.0,39,tf_idf_2gram,...,cosine,g=accent; gq=accent; qq=accent,610.7/732.0,100.0/112.0,19.51,19.55,19.42,19.49,0.05,"{'ass': 100, 'tam': 11, 'raj': 1}"
45,assamese[F],assamese[F]=1,150,20,FL2MI,phone_decay-0.5,NaN,1.0,39,tf_idf_2gram,...,cosine,g=accent; gq=accent; qq=accent,640.0/734.3,103.0/113.0,20.18,19.66,19.56,19.80,0.27,"{'ass': 103, 'tam': 9, 'kan': 1}"
22,assamese[F],assamese[F]=1,150,20,GCMI,phone_decay-1.0,NaN,1.0,39,tf_idf_2gram,...,cosine,g=accent; gq=accent; qq=accent,693.2/734.3,114.0/118.0,19.86,20.24,19.84,19.98,0.18,"{'ass': 114, 'tam': 4}"
24,assamese[F],assamese[F]=1,150,20,GCMI,phone_decay-0.2,NaN,1.0,39,tf_idf_2gram,...,cosine,g=accent; gq=accent; qq=accent,478.2/733.6,79.0/105.0,21.01,21.92,21.14,21.36,0.40,"{'ass': 79, 'tam': 18, 'guj': 6, 'kan': 1, 'ma..."
289,gujarati[F],gujarati[F]=1,150,20,FL2MI,phone_decay-0.2,NaN,1.0,39,tf_idf_2gram,...,cosine,g=accent; gq=accent; qq=accent,562.7/728.5,54.0/71.0,10.46,10.50,10.51,10.49,0.02,"{'guj': 54, 'tam': 12, 'kan': 2, 'ass': 2, 'ra..."
318,gujarati[F],gujarati[F]=1,150,20,FL2MI,phone_decay-1.0,NaN,1.0,39,tf_idf_2gram,...,cosine,g=accent; gq=accent; qq=accent,706.7/736.4,80.0/83.0,9.89,10.08,9.93,9.97,0.08,"{'guj': 80, 'tam': 3}"


In [14]:
filtered_df.to_csv("report-21st-oct2022.csv", index=False)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 10))
x = list(range(1, len(df["etaScale"]) + 1))
plt.errorbar(x, df["WER-mean"], df["WER-stdev"], marker="o", ecolor="red")
plt.xticks(x, df["etaScale"])
# plt.yscale('log')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 10))
x = list(range(1, len(df["etaScale"]) + 1))
plt.plot(x, df["WER-mean"])
plt.xticks(x, df["etaScale"])
# plt.yscale('log')
plt.show()


In [ ]:
# total selection : 100 100 100 -> 100.00
# total selection duration: 357.0149433106577 357.0149433106577 357.0149433106577 -> 357.01
# speakered selection: 76 76 76 -> 76.00
# speakered duration: 254.74947845804974 254.74947845804974 254.74947845804974 -> 254.75

# all selections: [Counter({'hindi': 76, 'korean': 8, 'spanish': 7, 'arabic': 3, 'chinese': 3, 'vietnamese': 3}), Counter({'hindi': 76, 'korean': 8, 'spanish': 7, 'arabic': 3, 'chinese': 3, 'vietnamese': 3}), Counter({'hindi': 76, 'korean': 8, 'spanish': 7, 'arabic': 3, 'chinese': 3, 'vietnamese': 3})]

# Evaluation Greedy WER: 16.19

df.to_csv(csv_name, index=False)


In [ ]:
# sample_path = 'Error-Driven-ASR-Personalization/CMU_expts/speaker/hindi/manifests/TSS_output/all/budget_100/target_50/FL1MI/eta_1.0/euclidean/39/stats.txt'
# CMU_expts/speaker_without/ABA/manifests/TSS_output/all/budget_100/target_50/FL1MI/eta_1.0/euclidean/39/run_1/
# budget = 100


budget = 150
# target = 50
ngram = 2
target = 20
# base_eta = "423.28"
# etaScales = [
#     "0.1",
#     "0.2",
#     "0.3",
#     "0.4",
#     "0.5",
#     "0.6",
#     "0.7",
#     "0.8",
#     "0.9",
#     "1.0",
#     "2.0",
#     "3.0",
#     "4.0",
#     "5.0",
#     "6.0",
#     "7.0",
#     "8.0",
#     "9.0",
#     "10.0",
# ]

# features = 'TRILL'
csv_name = "mod_report_{}_{}.csv".format(budget, target)

cols = [
    "speaker",
    "function",
    "base_eta",
    "etaScale",
    "target",
    # "accent_features",
    # "content_features",
    # "accent_similairty",
    # "content_similarity",
    "duration",
    "samples",
    "WER-r1",
    "WER-r2",
    "WER-r3",
    "WER-mean",
    "WER-stdev",
    "speakers",
]
df = pd.DataFrame(columns=cols)

speakers = [
    "assamese_female_english",
    # "manipuri_female_english",
    "kannada_male_english",
    # "rajasthani_male_english",
    # "hindi_male_english",
    # "malayalam_male_english",
    # "tamil_male_english",
    # "gujarati_female_english",
]


for speaker in speakers:
    if not (pathlib.Path(f"./{speaker}/all/budget_{budget}/").is_dir()):
        continue
    pick_from = "all"
    if not (pathlib.Path(f"./{speaker}/all/budget_{budget}/target_{target}/").is_dir()):
        continue
    for function in get_dirs(f"./{speaker}/all/budget_{budget}/target_{target}/"):
        (func, base_eta, etaScale) = split_function(function)
        for accent_features in get_dirs(
            f"./{speaker}/all/budget_{budget}/target_{target}/{function}/"
        ):
            for content_features in get_dirs(
                f"./{speaker}/all/budget_{budget}/target_{target}/{function}/{accent_features}"
            ):
                for accent_similarity in get_dirs(
                    f"./{speaker}/all/budget_{budget}/target_{target}/{function}/{accent_features}/{content_features}/"
                ):
                    for content_similarity in get_dirs(
                        f"./{speaker}/all/budget_{budget}/target_{target}/{function}/{accent_features}/{content_features}/{accent_similarity}"
                    ):
                        stats_file_path = f"./{speaker}/all/budget_{budget}/target_{target}/{function}/{accent_features}/{content_features}/{accent_similarity}/{content_similarity}/stats.txt"
                        if not (os.path.isfile(stats_file_path)):
                            continue
                        stats_file = open(stats_file_path, "r")
                        lines = stats_file.readlines()
                        # print("lines length ", len(lines), lines)
                        (
                            total_selections,
                            total_durations,
                            speakered_selections,
                            speakered_durations,
                        ) = map(get_each_run, lines[:4])
                        # print(total_selections, total_durations, speakered_selections, speakered_durations)
                        sample_frac = mean(
                            [
                                x[0] / x[1]
                                for x in zip(speakered_selections, total_selections)
                            ]
                        )
                        sample_total = mean(total_selections)
                        duration_frac = mean(
                            [
                                x[0] / x[1]
                                for x in zip(speakered_durations, total_durations)
                            ]
                        )
                        duration_total = mean(total_durations)
                        df_duration = "{:.2f}/{:.2f}".format(
                            duration_total * duration_frac, duration_total
                        )
                        df_samples = "{:.2f}/{:.2f}".format(
                            sample_total * sample_frac, sample_total
                        )
                        df_selections = get_selection_counts(lines[4])
                        try:
                            wers = [
                                WER_test_file(
                                    get_test_file_from_stats_path(i, stats_file)
                                )
                                for i in range(1, 4)
                            ]
                            df_wer_mean = round(mean(wers), 2)
                            df_wer_stdev = round(variance(wers), 3) ** 0.5
                            df = df.append(
                                dict(
                                    zip(
                                        cols,
                                        [
                                            speaker,
                                            func,
                                            base_eta,
                                            etaScale,
                                            target,
                                            # accent_features,
                                            # content_features,
                                            # accent_similarity,
                                            # content_similarity,
                                            df_duration,
                                            df_samples,
                                        ]
                                        + wers
                                        + [df_wer_mean, df_wer_stdev]
                                        + df_selections,
                                    )
                                ),
                                ignore_index=True,
                            )
                        except:
                            #                     continue
                            print(
                                "no WER's in file",
                                get_test_file_from_stats_path(1, stats_file),
                            )
                            wers = [0, 0, 0]
                            df_wer_mean = 0
                            df_wer_stdev = 0
                        # df = df.append(
                        #     dict(
                        #         zip(
                        #             cols,
                        #             [
                        #                 speaker,
                        #                 func,
                        #                 base_eta,
                        #                 etaScale,
                        #                 target,
                        #                 # accent_features,
                        #                 # content_features,
                        #                 # accent_similarity,
                        #                 # content_similarity,
                        #                 df_duration,
                        #                 df_samples,
                        #             ]
                        #             + wers
                        #             + [df_wer_mean, df_wer_stdev]
                        #             + df_selections,
                        #         )
                        #     ),
                        #     ignore_index=True,
                        # )
                        stats_file.close()
df = df.sort_values(
    by=[
        "speaker",
        # "accent_features",
        # "content_features",
        "function",
        "base_eta",
        "etaScale",
    ],
    ascending=True,
    ignore_index=True,
)
display(df)

In [ ]:
def get_unique_percent(json_file):
    with open(json_file) as file:
        lines = file.readlines()
    unq = set(lines)
    return len(unq)/len(lines)


# sample_path = 'Error-Driven-ASR-Personalization/CMU_expts/speaker/hindi/manifests/TSS_output/all/budget_100/target_50/FL1MI/eta_1.0/euclidean/39/stats.txt'
# CMU_expts/speaker_without/ABA/manifests/TSS_output/all/budget_100/target_50/FL1MI/eta_1.0/euclidean/39/run_1/
# budget = 100

## This cell is for the duplication report results


budget = 150
# target = 50
ngram = 2
target = 20
# base_eta = "423.28"
# etaScales = [
#     "0.1",
#     "0.2",
#     "0.3",
#     "0.4",
#     "0.5",
#     "0.6",
#     "0.7",
#     "0.8",
#     "0.9",
#     "1.0",
#     "2.0",
#     "3.0",
#     "4.0",
#     "5.0",
#     "6.0",
#     "7.0",
#     "8.0",
#     "9.0",
#     "10.0",
# ]

# features = 'TRILL'
csv_name = "mod_report_{}_{}.csv".format(budget, target)

cols = [
    "speaker",
    "function",
    "etaScale",
    "target",
    # "accent_features",
    # "content_features",
    # "accent_similairty",
    # "content_similarity",
    # "g_kernel",
    # "gq_kernel",
    # "qq_kernel",
    "duration",
    "samples",
    "WER-r1",
    "WER-r2",
    "WER-r3",
    "WER-mean",
    "WER-stdev",
    "speakers",
    "unique_percent",
]
df = pd.DataFrame(columns=cols)

speakers = [
    "assamese_female_english",
    "manipuri_female_english",
    "kannada_male_english",
    "rajasthani_male_english",
    "hindi_male_english",
    "malayalam_male_english",
    "tamil_male_english",
    "gujarati_female_english",
]

for speaker in speakers:
    if not (pathlib.Path(f"./{speaker}/all/budget_{budget}/").is_dir()):
        continue
    pick_from = "all"
    if not (pathlib.Path(f"./{speaker}/all/budget_{budget}/target_{target}/").is_dir()):
        continue
    for function in get_dirs(f"./{speaker}/all/budget_{budget}/target_{target}/"):
        if(len(function.split('_')) != 3): continue
        # print(function, function.split('_'), len(function.split('_')))
        (func, etaScale) = (function.split('_')[0], function.split('_')[2])
        for accent_features in get_dirs(
            f"./{speaker}/all/budget_{budget}/target_{target}/{function}/"
        ):
            if not accent_features.endswith("_3rep"):
                continue
            for content_features in get_dirs(
                f"./{speaker}/all/budget_{budget}/target_{target}/{function}/{accent_features}"
            ):
                for kernel_type in get_dirs(
                    f"./{speaker}/all/budget_{budget}/target_{target}/{function}/{accent_features}/{content_features}/"
                ):
                    for accent_similarity in get_dirs(
                        f"./{speaker}/all/budget_{budget}/target_{target}/{function}/{accent_features}/{content_features}/{kernel_type}"
                    ):
                        for content_similarity in get_dirs(
                            f"./{speaker}/all/budget_{budget}/target_{target}/{function}/{accent_features}/{content_features}/{kernel_type}/{accent_similarity}"
                        ):
                            json_file = f"./{speaker}/all/budget_{budget}/target_{target}/{function}/{accent_features}/{content_features}/{kernel_type}/{accent_similarity}/{content_similarity}/train.json"
                            unique_percent = get_unique_percent(json_file)
                            stats_file_path = f"./{speaker}/all/budget_{budget}/target_{target}/{function}/{accent_features}/{content_features}/{kernel_type}/{accent_similarity}/{content_similarity}/stats.txt"
                            if not (os.path.isfile(stats_file_path)):
                                continue
                            stats_file = open(stats_file_path, "r")
                            lines = stats_file.readlines()
                            # print("lines length ", len(lines), lines)
                            (
                                total_selections,
                                total_durations,
                                speakered_selections,
                                speakered_durations,
                            ) = map(get_each_run, lines[:4])
                            # print(total_selections, total_durations, speakered_selections, speakered_durations)
                            sample_frac = mean(
                                [
                                    x[0] / x[1]
                                    for x in zip(speakered_selections, total_selections)
                                ]
                            )
                            sample_total = mean(total_selections)
                            duration_frac = mean(
                                [
                                    x[0] / x[1]
                                    for x in zip(speakered_durations, total_durations)
                                ]
                            )
                            duration_total = mean(total_durations)
                            df_duration = "{:.2f}/{:.2f}".format(
                                duration_total * duration_frac, duration_total
                            )
                            df_samples = "{:.2f}/{:.2f}".format(
                                sample_total * sample_frac, sample_total
                            )
                            df_selections = get_selection_counts(lines[4])

                            wers = []
                            for i in range(1, 4):
                                try:
                                    wers.append(WER_test_file(get_test_file_from_stats_path(i, stats_file)))
                                except:
                                    print(
                                        "no WER's in file",
                                        get_test_file_from_stats_path(1, stats_file),
                                    )
                            
                            df_wer_mean = round(inf if len(wers) == 0  else mean(wers), 2)
                            df_wer_stdev = round(inf if len(wers) <= 1 else variance(wers), 3) ** 0.5
                            while(len(wers)<3): wers.append(0)
                            # print(wers, speaker, func)
                            
                            df = df.append(
                                dict(
                                    zip(
                                        cols,
                                        [
                                            speaker,
                                            func,
                                            etaScale,
                                            target,
                                            # accent_features,
                                            # content_features,
                                            # accent_similarity,
                                            # content_similarity,
                                            df_duration,
                                            df_samples,
                                        ]
                                        + wers
                                        + [df_wer_mean, df_wer_stdev]
                                        + df_selections
                                        + [unique_percent],
                                    )
                                ),
                                ignore_index=True,
                            )
                                # wers = [0, 0, 0]
                                # df_wer_mean = 0
                                # df_wer_stdev = 0
                            # df = df.append(
                            #     dict(
                            #         zip(
                            #             cols,
                            #             [
                            #                 speaker,
                            #                 func,
                            #                 base_eta,
                            #                 etaScale,
                            #                 target,
                            #                 # accent_features,
                            #                 # content_features,
                            #                 # accent_similarity,
                            #                 # content_similarity,
                            #                 df_duration,
                            #                 df_samples,
                            #             ]
                            #             + wers
                            #             + [df_wer_mean, df_wer_stdev]
                            #             + df_selections,
                            #         )
                            #     ),
                            #     ignore_index=True,
                            # )
                            stats_file.close()
df = df.sort_values(
    by=[
        "speaker",
        # "accent_features",
        # "content_features",
        "function",
        "etaScale",
    ],
    ascending=True,
    ignore_index=True,
)
display(df)


In [ ]:
df.to_csv("mod_rep_csv_150_20.csv")

In [ ]:
def get_unique_percent(json_file):
    with open(json_file) as file:
        lines = file.readlines()
    unq = set(lines)
    return len(unq)/len(lines)


# sample_path = 'Error-Driven-ASR-Personalization/CMU_expts/speaker/hindi/manifests/TSS_output/all/budget_100/target_50/FL1MI/eta_1.0/euclidean/39/stats.txt'
# CMU_expts/speaker_without/ABA/manifests/TSS_output/all/budget_100/target_50/FL1MI/eta_1.0/euclidean/39/run_1/
# budget = 100

## This cell is for the duplication report results


budget = 150
# target = 50
ngram = 2
target = 20
# base_eta = "423.28"
# etaScales = [
#     "0.1",
#     "0.2",
#     "0.3",
#     "0.4",
#     "0.5",
#     "0.6",
#     "0.7",
#     "0.8",
#     "0.9",
#     "1.0",
#     "2.0",
#     "3.0",
#     "4.0",
#     "5.0",
#     "6.0",
#     "7.0",
#     "8.0",
#     "9.0",
#     "10.0",
# ]

# features = 'TRILL'
csv_name = "mod_report_{}_{}.csv".format(budget, target)

cols = [
    "speaker",
    "function",
    "etaScale",
    "target",
    # "accent_features",
    # "content_features",
    # "accent_similairty",
    # "content_similarity",
    # "g_kernel",
    # "gq_kernel",
    # "qq_kernel",
    "duration",
    "samples",
    "WER-r1",
    "WER-r2",
    "WER-r3",
    "WER-mean",
    "WER-stdev",
    "speakers",
    "unique_percent",
]
df = pd.DataFrame(columns=cols)

speakers = [
    "assamese_female_english",
    "manipuri_female_english",
    "kannada_male_english",
    "rajasthani_male_english",
    "hindi_male_english",
    "malayalam_male_english",
    "tamil_male_english",
    "gujarati_female_english",
]

for speaker in speakers:
    if not (pathlib.Path(f"./{speaker}/all/budget_{budget}/").is_dir()):
        continue
    pick_from = "all"
    if not (pathlib.Path(f"./{speaker}/all/budget_{budget}/target_{target}/").is_dir()):
        continue
    for function in get_dirs(f"./{speaker}/all/budget_{budget}/target_{target}/"):
        if(len(function.split('_')) != 3): continue
        # print(function, function.split('_'), len(function.split('_')))
        (func, etaScale) = (function.split('_')[0], function.split('_')[2])
        for accent_features in get_dirs(
            f"./{speaker}/all/budget_{budget}/target_{target}/{function}/"
        ):
            # if not accent_features.endswith("_3rep"):
            #     continue
            for content_features in get_dirs(
                f"./{speaker}/all/budget_{budget}/target_{target}/{function}/{accent_features}"
            ):
                for kernel_type in get_dirs(
                    f"./{speaker}/all/budget_{budget}/target_{target}/{function}/{accent_features}/{content_features}/"
                ):
                    for accent_similarity in get_dirs(
                        f"./{speaker}/all/budget_{budget}/target_{target}/{function}/{accent_features}/{content_features}/{kernel_type}"
                    ):
                        for content_similarity in get_dirs(
                            f"./{speaker}/all/budget_{budget}/target_{target}/{function}/{accent_features}/{content_features}/{kernel_type}/{accent_similarity}"
                        ):
                            json_file = f"./{speaker}/all/budget_{budget}/target_{target}/{function}/{accent_features}/{content_features}/{kernel_type}/{accent_similarity}/{content_similarity}/train.json"
                            unique_percent = get_unique_percent(json_file)
                            stats_file_path = f"./{speaker}/all/budget_{budget}/target_{target}/{function}/{accent_features}/{content_features}/{kernel_type}/{accent_similarity}/{content_similarity}/stats.txt"
                            if not (os.path.isfile(stats_file_path)):
                                continue
                            stats_file = open(stats_file_path, "r")
                            lines = stats_file.readlines()
                            # print("lines length ", len(lines), lines)
                            (
                                total_selections,
                                total_durations,
                                speakered_selections,
                                speakered_durations,
                            ) = map(get_each_run, lines[:4])
                            # print(total_selections, total_durations, speakered_selections, speakered_durations)
                            sample_frac = mean(
                                [
                                    x[0] / x[1]
                                    for x in zip(speakered_selections, total_selections)
                                ]
                            )
                            sample_total = mean(total_selections)
                            duration_frac = mean(
                                [
                                    x[0] / x[1]
                                    for x in zip(speakered_durations, total_durations)
                                ]
                            )
                            duration_total = mean(total_durations)
                            df_duration = "{:.2f}/{:.2f}".format(
                                duration_total * duration_frac, duration_total
                            )
                            df_samples = "{:.2f}/{:.2f}".format(
                                sample_total * sample_frac, sample_total
                            )
                            df_selections = get_selection_counts(lines[4])

                            wers = []
                            for i in range(1, 4):
                                try:
                                    wers.append(WER_test_file(get_test_file_from_stats_path(i, stats_file)))
                                except:
                                    print(
                                        "no WER's in file",
                                        get_test_file_from_stats_path(1, stats_file),
                                    )
                            
                            df_wer_mean = round(inf if len(wers) == 0  else mean(wers), 2)
                            df_wer_stdev = round(inf if len(wers) <= 1 else variance(wers), 3) ** 0.5
                            while(len(wers)<3): wers.append(0)
                            # print(wers, speaker, func)
                            
                            df = df.append(
                                dict(
                                    zip(
                                        cols,
                                        [
                                            speaker,
                                            func,
                                            etaScale,
                                            target,
                                            # accent_features,
                                            # content_features,
                                            # accent_similarity,
                                            # content_similarity,
                                            df_duration,
                                            df_samples,
                                        ]
                                        + wers
                                        + [df_wer_mean, df_wer_stdev]
                                        + df_selections
                                        + [unique_percent],
                                    )
                                ),
                                ignore_index=True,
                            )
                                # wers = [0, 0, 0]
                                # df_wer_mean = 0
                                # df_wer_stdev = 0
                            # df = df.append(
                            #     dict(
                            #         zip(
                            #             cols,
                            #             [
                            #                 speaker,
                            #                 func,
                            #                 base_eta,
                            #                 etaScale,
                            #                 target,
                            #                 # accent_features,
                            #                 # content_features,
                            #                 # accent_similarity,
                            #                 # content_similarity,
                            #                 df_duration,
                            #                 df_samples,
                            #             ]
                            #             + wers
                            #             + [df_wer_mean, df_wer_stdev]
                            #             + df_selections,
                            #         )
                            #     ),
                            #     ignore_index=True,
                            # )
                            stats_file.close()
df = df.sort_values(
    by=[
        "speaker",
        # "accent_features",
        # "content_features",
        "function",
        "etaScale",
    ],
    ascending=True,
    ignore_index=True,
)
display(df)


In [ ]:
df.to_csv("mix_query_set.csv", index=False)

In [ ]:
def get_unique_percent(json_file):
    with open(json_file) as file:
        lines = file.readlines()
    unq = set(lines)
    return len(unq)/len(lines)


# sample_path = 'Error-Driven-ASR-Personalization/CMU_expts/speaker/hindi/manifests/TSS_output/all/budget_100/target_50/FL1MI/eta_1.0/euclidean/39/stats.txt'
# CMU_expts/speaker_without/ABA/manifests/TSS_output/all/budget_100/target_50/FL1MI/eta_1.0/euclidean/39/run_1/
# budget = 100

## This cell is for the duplication report results


budget = 150
# target = 50
ngram = 2
target = 20
# base_eta = "423.28"
# etaScales = [
#     "0.1",
#     "0.2",
#     "0.3",
#     "0.4",
#     "0.5",
#     "0.6",
#     "0.7",
#     "0.8",
#     "0.9",
#     "1.0",
#     "2.0",
#     "3.0",
#     "4.0",
#     "5.0",
#     "6.0",
#     "7.0",
#     "8.0",
#     "9.0",
#     "10.0",
# ]

# features = 'TRILL'
csv_name = "mod_report_{}_{}.csv".format(budget, target)

cols = [
    "speaker",
    "function",
    "etaScale",
    "target",
    # "accent_features",
    # "content_features",
    # "accent_similairty",
    # "content_similarity",
    # "g_kernel",
    # "gq_kernel",
    # "qq_kernel",
    "duration",
    "samples",
    "WER-r1",
    "WER-r2",
    "WER-r3",
    "WER-mean",
    "WER-stdev",
    "speakers",
    "unique_percent",
]
df = pd.DataFrame(columns=cols)

speakers = [
    "assamese_female_english",
    "manipuri_female_english",
    "kannada_male_english",
    "rajasthani_male_english",
    "hindi_male_english",
    "malayalam_male_english",
    "tamil_male_english",
    "gujarati_female_english",
]

for speaker in speakers:
    if not (pathlib.Path(f"./{speaker}/all/budget_{budget}/").is_dir()):
        continue
    pick_from = "all"
    if not (pathlib.Path(f"./{speaker}/all/budget_{budget}/target_{target}/").is_dir()):
        continue
    for function in get_dirs(f"./{speaker}/all/budget_{budget}/target_{target}/"):
        if(len(function.split('_')) != 3): continue
        # print(function, function.split('_'), len(function.split('_')))
        (func, etaScale) = (function.split('_')[0], function.split('_')[2])
        if etaScale != "1.0":
            continue
        for accent_features in get_dirs(
            f"./{speaker}/all/budget_{budget}/target_{target}/{function}/"
        ):
            # if not accent_features.endswith("_3rep"):
            #     continue
            for content_features in get_dirs(
                f"./{speaker}/all/budget_{budget}/target_{target}/{function}/{accent_features}"
            ):
                for kernel_type in get_dirs(
                    f"./{speaker}/all/budget_{budget}/target_{target}/{function}/{accent_features}/{content_features}/"
                ):
                    for accent_similarity in get_dirs(
                        f"./{speaker}/all/budget_{budget}/target_{target}/{function}/{accent_features}/{content_features}/{kernel_type}"
                    ):
                        for content_similarity in get_dirs(
                            f"./{speaker}/all/budget_{budget}/target_{target}/{function}/{accent_features}/{content_features}/{kernel_type}/{accent_similarity}"
                        ):
                            json_file = f"./{speaker}/all/budget_{budget}/target_{target}/{function}/{accent_features}/{content_features}/{kernel_type}/{accent_similarity}/{content_similarity}/train.json"
                            unique_percent = get_unique_percent(json_file)
                            stats_file_path = f"./{speaker}/all/budget_{budget}/target_{target}/{function}/{accent_features}/{content_features}/{kernel_type}/{accent_similarity}/{content_similarity}/stats.txt"
                            if not (os.path.isfile(stats_file_path)):
                                continue
                            stats_file = open(stats_file_path, "r")
                            lines = stats_file.readlines()
                            # print("lines length ", len(lines), lines)
                            (
                                total_selections,
                                total_durations,
                                speakered_selections,
                                speakered_durations,
                            ) = map(get_each_run, lines[:4])
                            # print(total_selections, total_durations, speakered_selections, speakered_durations)
                            sample_frac = mean(
                                [
                                    x[0] / x[1]
                                    for x in zip(speakered_selections, total_selections)
                                ]
                            )
                            sample_total = mean(total_selections)
                            duration_frac = mean(
                                [
                                    x[0] / x[1]
                                    for x in zip(speakered_durations, total_durations)
                                ]
                            )
                            duration_total = mean(total_durations)
                            df_duration = "{:.2f}/{:.2f}".format(
                                duration_total * duration_frac, duration_total
                            )
                            df_samples = "{:.2f}/{:.2f}".format(
                                sample_total * sample_frac, sample_total
                            )
                            df_selections = get_selection_counts(lines[4])

                            wers = []
                            for i in range(1, 4):
                                try:
                                    wers.append(WER_test_file(get_test_file_from_stats_path(i, stats_file)))
                                except:
                                    print(
                                        "no WER's in file",
                                        get_test_file_from_stats_path(1, stats_file),
                                    )
                            
                            df_wer_mean = round(inf if len(wers) == 0  else mean(wers), 2)
                            df_wer_stdev = round(inf if len(wers) <= 1 else variance(wers), 3) ** 0.5
                            while(len(wers)<3): wers.append(0)
                            # print(wers, speaker, func)
                            
                            df = df.append(
                                dict(
                                    zip(
                                        cols,
                                        [
                                            speaker,
                                            func,
                                            etaScale,
                                            target,
                                            # accent_features,
                                            # content_features,
                                            # accent_similarity,
                                            # content_similarity,
                                            df_duration,
                                            df_samples,
                                        ]
                                        + wers
                                        + [df_wer_mean, df_wer_stdev]
                                        + df_selections
                                        + [unique_percent],
                                    )
                                ),
                                ignore_index=True,
                            )
                                # wers = [0, 0, 0]
                                # df_wer_mean = 0
                                # df_wer_stdev = 0
                            # df = df.append(
                            #     dict(
                            #         zip(
                            #             cols,
                            #             [
                            #                 speaker,
                            #                 func,
                            #                 base_eta,
                            #                 etaScale,
                            #                 target,
                            #                 # accent_features,
                            #                 # content_features,
                            #                 # accent_similarity,
                            #                 # content_similarity,
                            #                 df_duration,
                            #                 df_samples,
                            #             ]
                            #             + wers
                            #             + [df_wer_mean, df_wer_stdev]
                            #             + df_selections,
                            #         )
                            #     ),
                            #     ignore_index=True,
                            # )
                            stats_file.close()
df = df.sort_values(
    by=[
        "speaker",
        # "accent_features",
        # "content_features",
        "function",
        "etaScale",
    ],
    ascending=True,
    ignore_index=True,
)
display(df)


In [ ]:
print(os.path.sep)

In [ ]:

#     if not (pathlib.Path(f"./{speaker}/all/budget_{budget}/").is_dir()):
#         continue
#     pick_from = "all"
#     if not (pathlib.Path(f"./{speaker}/all/budget_{budget}/target_{target}/").is_dir()):
#         continue
#     for function in get_dirs(f"./{speaker}/all/budget_{budget}/target_{target}/"):
#         (func, base_eta, etaScale) = split_function(function)
#         for accent_features in get_dirs(
#             f"./{speaker}/all/budget_{budget}/target_{target}/{function}/"
#         ):
#             for content_features in get_dirs(
#                 f"./{speaker}/all/budget_{budget}/target_{target}/{function}/{accent_features}"
#             ):
#                 for accent_similarity in get_dirs(
#                     f"./{speaker}/all/budget_{budget}/target_{target}/{function}/{accent_features}/{content_features}/"
#                 ):
#                     for content_similarity in get_dirs(
#                         f"./{speaker}/all/budget_{budget}/target_{target}/{function}/{accent_features}/{content_features}/{accent_similarity}"
#                     ):
#                         stats_file_path = f"./{speaker}/all/budget_{budget}/target_{target}/{function}/{accent_features}/{content_features}/{accent_similarity}/{content_similarity}/stats.txt"
#                         if not (os.path.isfile(stats_file_path)):
#                             continue
#                         stats_file = open(stats_file_path, "r")
#                         lines = stats_file.readlines()
#                         # print("lines length ", len(lines), lines)
#                         (
#                             total_selections,
#                             total_durations,
#                             speakered_selections,
#                             speakered_durations,
#                         ) = map(get_each_run, lines[:4])
#                         # print(total_selections, total_durations, speakered_selections, speakered_durations)
#                         sample_frac = mean(
#                             [
#                                 x[0] / x[1]
#                                 for x in zip(speakered_selections, total_selections)
#                             ]
#                         )
#                         sample_total = mean(total_selections)
#                         duration_frac = mean(
#                             [
#                                 x[0] / x[1]
#                                 for x in zip(speakered_durations, total_durations)
#                             ]
#                         )
#                         duration_total = mean(total_durations)
#                         df_duration = "{:.2f}/{:.2f}".format(
#                             duration_total * duration_frac, duration_total
#                         )
#                         df_samples = "{:.2f}/{:.2f}".format(
#                             sample_total * sample_frac, sample_total
#                         )
#                         df_selections = get_selection_counts(lines[4])
#                         try:
#                             wers = [
#                                 WER_test_file(
#                                     get_test_file_from_stats_path(i, stats_file)
#                                 )
#                                 for i in range(1, 4)
#                             ]
#                             df_wer_mean = round(mean(wers), 2)
#                             df_wer_stdev = round(variance(wers), 3) ** 0.5
#                             df = df.append(
#                                 dict(
#                                     zip(
#                                         cols,
#                                         [
#                                             speaker,
#                                             func,
#                                             base_eta,
#                                             etaScale,
#                                             target,
#                                             # accent_features,
#                                             # content_features,
#                                             # accent_similarity,
#                                             # content_similarity,
#                                             df_duration,
#                                             df_samples,
#                                         ]
#                                         + wers
#                                         + [df_wer_mean, df_wer_stdev]
#                                         + df_selections,
#                                     )
#                                 ),
#                                 ignore_index=True,
#                             )
#                         except:
#                             #                     continue
#                             print(
#                                 "no WER's in file",
#                                 get_test_file_from_stats_path(1, stats_file),
#                             )
#                             wers = [0, 0, 0]
#                             df_wer_mean = 0
#                             df_wer_stdev = 0
#                         # df = df.append(
#                         #     dict(
#                         #         zip(
#                         #             cols,
#                         #             [
#                         #                 speaker,
#                         #                 func,
#                         #                 base_eta,
#                         #                 etaScale,
#                         #                 target,
#                         #                 # accent_features,
#                         #                 # content_features,
#                         #                 # accent_similarity,
#                         #                 # content_similarity,
#                         #                 df_duration,
#                         #                 df_samples,
#                         #             ]
#                         #             + wers
#                         #             + [df_wer_mean, df_wer_stdev]
#                         #             + df_selections,
#                         #         )
#                         #     ),
#                         #     ignore_index=True,
#                         # )
#                         stats_file.close()
# df = df.sort_values(
#     by=[
#         "speaker",
#         # "accent_features",
#         # "content_features",
#         "function",
#         "base_eta",
#         "etaScale",
#     ],
#     ascending=True,
#     ignore_index=True,
# )
# display(df)


In [ ]:
speakers = [
    "assamese_female_english",
    "manipuri_female_english",
    "kannada_male_english",
    "rajasthani_male_english",
    "hindi_male_english",
    "malayalam_male_english",
    "tamil_male_english",
    "gujarati_female_english",
]

for speaker in speakers:
    selection_json = f"./{speaker}/selection.json"
    print(speaker, end=" ")
    with open(selection_json) as file:
        print(len(file.readlines()))
    